# INFO 2950 Project Phase 4

#### Talia Boehm, Lucy Barsanti, Emma Shen, Maddi Klancher


Link to dataset: https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import seaborn
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression

In [2]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

# **Research Question:** 
* **Main Question**: How did the COVID-19 pandemic affect airline delays?

* **Supporting Questions**: What is the relationship between the COVID-19 rate and airline delays in the United States between January 2020 - July 2022? Does the relationship differ between states? By airline?

# **Data Cleaning:**

**Airline Delays Dataset**

* The data was exported as a CSV file. We did not remove any columns so that we could analyze all the different types of delays as well as geographic and carrier information. We basically did our “cleaning” by filtering on the BTS website to find flight delay information from January 2017 - July 2022.

**COVID-19 Cases Dataset**

* The data was exported as a CSV file. We removed unnecessary columns and added a couple of new columns to make our analysis easier. In addition, we compressed certain rows based on state, month, and year to ensure that our data were consistent with the airline delay dataset. The initial dataset was organized by day. A lot of these edits were made so as to ensure it was a fully usable dataset, and as well to ensure that there were no values that were null or would affect our analysis work. More specifically, our cleaning focused on:

* Converting objects to DateTime: This was a significant step, because much of our analysis required us to pull dates from the datasets. Unfortunately, the dates were not formatted within the CSV as DateTime objects, and thus our job was simply to transform these dates into DateTime objects. This was done using the pandas to_datetime() function, and assisted the team immensely in being able to directly pull the dates/times in a usable form for our data analysis. By being able to have these dates in DateTime objects, we are then able to use them without issue (in comparison to strings or other types) within our data analysis. 

* Adding month and year columns: In order to join the datasets by state, we needed to create new columns for the year and month of each record. This was simple since we already converted dates into DateTime objects. By adding these two columns, we were able to sum the covid cases and group by state, year, and month. 

* Removing all attributes except year	, month, state, tot_case, and new_case: We made the conscious decision to only use year, month, state, total cases, and new cases. We decided to omit the rest because our research question only pertains to COVID-19 cases, and not deaths. Moreover, we are not concerned with probable cases as the data is subjective based on whether or not consent was given to include those metrics. As a result, we decided to stick only with objective data for our analysis.

# Data Description

## Flight delay data from January 2017 - July 2022:

* This dataset contains information on flights in the United States from January 2017 to July 2022. The data contains attributes including the flight date, flight number, origin airport, destination airport, departure time, arrival time, and the reason for any delays. There are 7 reasons for flight delays outlined by the dataset: air carrier delay, weather delay, national aviation system delay, security delay, aircraft arriving late, canceled, and diverted. 

* Raw data can be found [here](https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp) 
* Link to our [Google Drive](https://drive.google.com/file/d/1Ci4XzBpg3BF-_zx70sBTN1uVkaERmTgl/view?usp=sharing)

* **Observations**: Each observation in the dataset includes the number of occurrences of each cause of delay for one airline at one airport for one month. For example, one row in the dataset represents the flight delay statistics for American Airlines at Nashville International Airport for the month of July 2022. 

* **Attributes**: The attributes of this dataset are as follows:

**year:** Year data collected\
**month:** Numeric representation of the month\
**carrier:** Airline carrier number\
**carrier_name:** Airline carrer\
**airport:** Official airport code\
**airport_name:** Full airport name\
**arr_flights:** Number of flights arriving at airport\
**arr_del15:** Number of flights more than 15 minutes late\
**carrier_ct:** Number of flights delayed due to air carrier. (e.g. no crew)\
**weather_ct:** Number of flights due to weather\
**nas_ct:** Number of flights delayed due to National Aviation System\
**security_ct:** Number of flights canceled due to a security breach\
**late_aircraft_ct:** Number of flights delayed as a result of another flight on the same aircraft delayed\
**arr_cancelled:** Total number of canceled flights\
**arr_diverted:** Total number of diverted flights\
**arr_delay:** Total number of minutes of delay for arriving flights\
**carrier_delay:** Total number of minutes of delay due to carrier delays\
**weather_delay:** Total number of minutes of delay due to weather delays\
**nas_delay:** Total number of minutes of delay due to National Aviation System delays\
**security_delay:** Total number of minutes of delay due to security delays\
**late_aircraft_delay:** Total time (minutes) of delay as a result of a previous flight on the same airplane being late

* The U.S. Department of Transportation (DOT) created and funded the monthly Air Travel Consumer Report to provide information to the public about the quality of air travel. In response, the Bureau of Transportation Statistics (BTS) began collecting details on the causes of flight delays in June 2003. The air travel data found on the BTS website is publicly available and required by the U.S. Congress.

* Operating carriers that have a 0.5 percent of total domestic scheduled-service passenger revenue are required to report on-time data and the causes of delay. Our data comes from these 17 operating carriers reporting non-stop scheduled-service flights between points within the United States and including territories. The DOT’s framework determined the reporting categories to ensure the public would have clear information about the nature and sources of airline delays and cancellations. The Bureau of Transportation Statistics conducted a pilot program to test the monthly reporting of causation and then implemented this framework in 2001.

* The DOT defines a “delayed” flight if it operated more than 15 minutes later than the scheduled arrival time. As a result, the data on delayed flights abides by this definition. 

* We downloaded the data from the BTS website as a CSV file.

## COVID-19 cases and deaths by state over time from January 22, 2020 - October 20, 2022:

* The raw data can be found [here](https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36/data)

* Link to our [Google Drive](https://drive.google.com/file/d/1i3fQ9wMkgdViOwuJO_EKLR2p1Hd_UCyF/view?usp=sharing)

* This dataset contains archived aggregate daily counts of COVID-19 cases and death by state. 

* **Observations**: Each observation contains archived aggregate daily counts of COVID-19 cases and death for each state in the United States. For example, we can see one row that contains data from 10/18/22 for the state of New York with 3,275,131 total cases, 1,660 new cases, 29,356 total deaths, 32 new deaths, and 0 new probable deaths.

* **Attributes**: The attributes of this dataset are as follows:

**submission_date:** date of counts\
**state:** jurisdiction\
**tot_cases:** total number of cases\
**conf_cases:** total confirmed cases\
**prob_cases:** total probable cases\
**new_case:** number of new cases\
**pnew_case:** number of new probable cases\
**tot_death:** total number of deaths\
**conf_death:** total confirmed deaths\
**prob_death:** total probable deaths\
**new_death:** number of new deaths\
**pnew_death:** number of new probable deaths\
**created_at:** date and time record was created\
**consent_cases:** if agree, then confirmed and probable cases are included\
**consent_deaths:** if agree, then confirmed and probable deaths are included\
**year:** year or counts\
**month:** month of counts


* The Centers for Disease Control and Prevention (CDC) made the United States COVID-19 Cases and Deaths by State over Time dataset to track the spread of COVID-19 in the United States. The United States government funded this dataset. 

* We are using the data to compare flight delay information with COVID-19 cases on a monthly basis based on state to determine if there is any relationship between the two datasets.

* The CDC retrieves aggregate case and death data from jurisdictional and state partners. However, counting the exact number of COVID-19 cases is not possible due to a variety of reasons. For instance, people do not always report their illnesses, symptoms might not appear immediately, and there may be delays in testing. Moreover, there are sometimes false test results. Additionally, each state may have different testing and reporting protocols, population sentiment toward COVID-19, and levels of testing efficiency.

* We downloaded the data from the CDC’s website as a CSV file.

# Data Limitation

#### **First Data Set (Flight Delays)**:

* The data is somewhat limited because it does not include all flight data for the specified time period. Although the US has ​​19,622 airports, the data only includes information about 384 airports. Another limitation is that the data does not cover delays experienced by flights operated by US-based airlines but originating or arriving at airports outside the US. Although we are looking at the relationship between COVID-19 cases and delays in the US, international statistics could potentially affect our data too.

#### **Second Data Set (COVID-19 Cases)**:

* There are several limitations to any COVID-19 dataset. First, there is a lot of noise and misinformation surrounding the pandemic, making it difficult to separate fact from fiction. Secondly, different jurisdictions handle their data differently, making it difficult to get a complete picture of the pandemic within the United States. The data may also be affected by political and geographical demographics.

# Exploratory Data Analysis 

## 1. Flight Delays Dataset
_Note: For a detailed description of this dataset, please see the section "Data Description" above._

The following cells show a preview of the original Flight Delay dataframe.
•	This dataframe is stored in the variable df_delays


In [3]:
df_delays = pd.read_csv('Airline_Delay_Cause.csv')
df_delays.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2022,7,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",33.0,2.0,0.92,1.00,...,0.0,0.00,0.0,0.0,129.0,98.0,23.0,8.0,0.0,0.0
1,2022,7,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",78.0,25.0,11.80,0.72,...,0.0,7.48,0.0,0.0,1664.0,887.0,52.0,224.0,0.0,501.0
2,2022,7,9E,Endeavor Air Inc.,ACK,"Nantucket, MA: Nantucket Memorial",124.0,19.0,5.84,1.00,...,0.0,5.40,5.0,4.0,1523.0,388.0,35.0,511.0,0.0,589.0
3,2022,7,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",67.0,10.0,1.32,1.00,...,1.0,4.28,0.0,1.0,657.0,103.0,82.0,93.0,25.0,354.0
4,2022,7,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",174.0,30.0,18.10,5.75,...,0.0,2.55,1.0,0.0,2462.0,1686.0,310.0,139.0,0.0,327.0


### Inserting new column "state"
By parsing each value in the column __airport_name__, we extracted the 2-character state abbrevaition for each row of data and created a new column __state__.

_Rationale for change:_ As you will see in the next section, the COVID dataset has a column- state- containing the state pertinent to each row of data. To combine the COVID and Flight Delay for specific stats, we needed to insert the column state into df_delays. This addition allows us to join the 2 datasets on the __state__ column.

In the preview of df_delays below, you can see the new column titled __state__


In [4]:
airports= df_delays['airport_name']
state = []
for x in airports:
    idx = x.rindex(":")
    start = idx - 2
    state_x = x[start: idx]
    state.append(state_x)

df_delays['state'] = state
df_delays.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,state
0,2022,7,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",33.0,2.0,0.92,1.00,...,0.00,0.0,0.0,129.0,98.0,23.0,8.0,0.0,0.0,PA
1,2022,7,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",78.0,25.0,11.80,0.72,...,7.48,0.0,0.0,1664.0,887.0,52.0,224.0,0.0,501.0,GA
2,2022,7,9E,Endeavor Air Inc.,ACK,"Nantucket, MA: Nantucket Memorial",124.0,19.0,5.84,1.00,...,5.40,5.0,4.0,1523.0,388.0,35.0,511.0,0.0,589.0,MA
3,2022,7,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",67.0,10.0,1.32,1.00,...,4.28,0.0,1.0,657.0,103.0,82.0,93.0,25.0,354.0,LA
4,2022,7,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",174.0,30.0,18.10,5.75,...,2.55,1.0,0.0,2462.0,1686.0,310.0,139.0,0.0,327.0,GA


## 2. COVID Data by State

_Note: For a detailed description of this dataset, please see the section "Data Description" above._

The following cells show a preview of the dataframe stored in the variable __df_covid__. The dataset contains information about COVID cases over time in each US State.

In [5]:
df_covid = pd.read_csv('Covid-by-State.csv')
df_covid['submission_date']= pd.to_datetime(df_covid['submission_date'])
df_covid['year'] = pd.DatetimeIndex(df_covid['submission_date']).year
df_covid['month'] = pd.DatetimeIndex(df_covid['submission_date']).month
df_covid.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Covid-by-State.csv'

### Cleaning the COVID Dataset

Steps taken to clean __df_covid__:
- Summing COVID data for each MONTH
- Grouping and ordering by: state, month, and year
    - The flight delay dataset and the COVID dataset have these 3 columns in common, so this is how we will group and order our data when combining __df_covid__ and __df_delays__
    - This ordering is also logical because the year/ month increases as you travel down the rows in the dataset, and the states are ordered alphabetically (A-Z)


In [ ]:

%sql df_covid_cleaned << SELECT year, month, state, SUM(tot_cases) AS tot_case, SUM(new_case) AS new_case FROM df_covid GROUP BY state, month, year ORDER BY year, month, state ASC

df_covid_cleaned


In [ ]:
## Lucy
dates = df_covid_cleaned[['year', 'month']]
month_date = []

for x in dates.index:
    if dates['month'][x] <= 9:
        month_date_x = (dates['year'][x]) + (dates['month'][x] / 12)
    else:
        month_date_x = (dates['year'][x]) + (dates['month'][x] / 12)
    
    month_date.append(float(month_date_x))
    
df_covid_cleaned['year_month'] = month_date

df_covid_cleaned.head()


## Visualization 1: New COVID Cases by State

This visualaization uses a Log Scale to show how COVID cases increased in each state over time. 

### Insights

- COVID cases skyrocketed at the beginning months of 2020
- COVID cases have remained high from 5/2020 to 7/2022 despite fluctuations.
- Trend lines for each state are parallel to one another, suggesting that the rate of new infections has changed at similar rates across state lines. 

In [ ]:
## Lucy
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

plt.yscale('log')
plt.title("New Covid Cases by State (Log Scale)")
plt.xlabel("Year")
plt.ylabel("Log (New Cases)")

for state in states:
    state_data_find = pd.Series(df_covid_cleaned['state'] == state)
    
    state_data = df_covid_cleaned.loc[state_data_find]
    x = state_data['year_month']
    y = state_data['new_case']
    
    plt.plot(x, y)

##print(ak_data)

In [ ]:
## MADDI
%sql Endeavor_ABE_22_df << SELECT * FROM df_delays WHERE carrier = '9E' AND airport = 'ABE' AND year = 2022
Endeavor_ABE_22_df.head()

## Visualization 2: Endeavor Air Inc. Delays at ABE from Jan-Jul 2022

This is a lineplot showing the number of Endeavor Airlines flight delays at airport ABE (Allentown/Bethlehem/ Easton) over time.

Since we are not familiar with airline delay data, we wanted to see how delays change over a short period of time. 


### Insights:
- There is major fluctuation in delays over a period this short for the given airline and airport
    - The number of delays is fairly low for this airport
- The number of delayed flights may not be an effective metric given the differences in sizes across airports. We will likely use the percentage of total fights delayed when comparing/analyzing data from more than one airport.

In [ ]:
## MADDI
seaborn.lineplot(data=Endeavor_ABE_22_df, x="month", y="arr_del15").set(title='Endeavor Air Inc. Delays at ABE from Jan-Jul 2022')

## Visualization 3: Delta Airline Delays at LAX by month

To better understand flight delay trends from 2020-2022, we plotted the total # of delays on Delta Airlines at the Los Angeles Airport (LAX) over the past 2 years. 

In [ ]:
## MADDI
%sql Delta_LAX_Jan_df << SELECT * FROM df_delays WHERE carrier = 'DL' AND airport = 'LAX'
Delta_LAX_Jan_df.head()

dates = Delta_LAX_Jan_df[['year', 'month']]
month_date = []

for x in dates.index:
    if dates['month'][x] <= 9:
        month_date_x = (dates['year'][x]) + (dates['month'][x] / 12)
    else:
        month_date_x = (dates['year'][x]) + (dates['month'][x] / 12)
    
    month_date.append(float(month_date_x))
    
Delta_LAX_Jan_df['year_month'] = month_date

Delta_LAX_Jan_df.head()

### Insights 

- Shown in the graph below, there seems to be a general increase in the number of delayed flights over time. 

- In both the previous graph of Endeavor Airlines, and in the graph below, it's important to note that despite an overall increase in delays, there is significant fluctuation in delays month-month. 
- Delta at LAX has a lot more delays than Endeavor Air at ABE
    - This reinforces that we should use percent of total flights delayed instead of counts due to variation between airports


In [ ]:
## MADDI
seaborn.lineplot(data=Delta_LAX_Jan_df, x="year_month", y="arr_del15").set(title='Delta Air Lines Delays at LAX by Month')
plt.xlabel('Year')
plt.ylabel('Number of Delayed Flights')
plt.xticks(rotation=60)

In [ ]:
## MADDI
%sql JFK_Jul_2022_df << SELECT * FROM df_delays WHERE airport = 'JFK' AND year = 2022 AND month = 7
JFK_Jul_2022_df.head()

In [ ]:
seaborn.barplot(data=JFK_Jul_2022_df, x="carrier_name", y="arr_del15").set(title='Delays by Airline at JFK for July 2022')
plt.xticks(rotation=60)

In [ ]:
df_delays.describe()

In [ ]:
df_delays.corr()

In [ ]:
df_delays.cov()

In [ ]:
## MADDI

%sql year20_df << SELECT * FROM df_delays WHERE year = 2020
year20_df.head()

In [ ]:
# scatterplot for x1 and y1
seaborn.regplot(data = year20_df, x='month', y='arr_del15')
plt.xlabel("Month")
plt.ylabel("Number of Delays")
plt.title("Number of Delays in 2020")

In [ ]:
df_delays["percent_delayed"] = df_delays["arr_del15"]/df_delays["arr_flights"]
df_delays["percent_cancelled"] = df_delays["arr_cancelled"]/df_delays["arr_flights"]
df_delays.head()

In [ ]:
## LUCY
%sql df_covid_flight << SELECT df_delays.*, df_covid_cleaned.tot_case, df_covid_cleaned.new_case, df_covid_cleaned.year_month FROM df_delays INNER JOIN df_covid_cleaned ON df_delays.month = df_covid_cleaned.month AND df_delays.year = df_covid_cleaned.year AND df_delays.State = df_covid_cleaned.state

df_covid_flight

In [ ]:
ny_data_find = pd.Series(df_covid_flight['state'] == 'NY')
    
ny_data = df_covid_flight.loc[ny_data_find]

ny_data.head()

In [ ]:
%sql df_ny_covid_flight << SELECT SUM(new_case) as new_cases, SUM(arr_del15) as delayed_flights FROM ny_data GROUP BY year_month

df_ny_covid_flight.head()

## Visualization 7

This visualization of Delayed Flights vs. New COVID Cases for New York shows us that there is not a strong relationship between the number of delayed flights and the number of new covid cases at first glance. We will have to do some further data exploration to try to understand why.

In [ ]:
ny_x = df_ny_covid_flight['delayed_flights']
ny_y = df_ny_covid_flight['new_cases']

plt.scatter(ny_x, ny_y)
plt.xlabel("Delayed Flights")
plt.ylabel("New COVID Cases")
plt.title("Delayed Flights vs. New COVID Cases in NY")
plt.show()

## Questions for reviewers 
1. Should we consider population of a given state when analyzing our data?
2. Should we consider population of a given state when analyzing our data?

 If you think there is some interesting analyses that you could incorporate if you use population data, then yes! Looking at your data right now, I could population data maybe be helpful for a percentage statistic (what percentage of a state population had covid in a given time period. It could also be helpful to scale covid cases using population of state because larger states would most likely have more covid cases. I think including it could add a layer of complexity and yield interesting statistics, but I dont think it is necessary.
 
3. Should we format all graph date axes the same way or is it okay if we specify differences in the header?

It is fine to have different formats as long as the scale you are using makes sense based on the data and it is clear from the title and axis labels what you are plotting. 

4. Are our variable names written in an understandeable manner?
The column names look fine to me, I would note that when you plot them it would be helpful to change the name to something more immediately understandable. For example, in your graph for Delays by Airline at JFK for July 2022, the axis label arr_del15 could be changed to something more informative, but using your current column names for analyses is completely acceptable.

5. Since the COVID dataset contains monthly totals for each state, should we group together airports in the same state for flight delays or evaluate them individually?
I think it might be interesting to do both. I wouldnt only group together airports in the same state, but this could be an interesting additional. One thing that concerns me slightly is that not all the people in an airport would test positive for covid within the state that the airport is located in.
 
6. Are there other relationships for the vars that we should be evaluating?
I think your EDA was pretty encompassing regarding the flight dataset, but I think it would be worth exploring more relationships between the two datasets you have, especially since it concerns your research question. For example, I think it could be interesting to look into arr_delay to see what the relationship is between covid and the actual number of minutes the flight was delayed.

### Preregistration of Analyses ###

1. Comparing how the relationship between the COVID-19 rate and the number of airline delays in the U.S between January 2022-July 2022 varies across different states.
2. Comparing how the relationship between the COVID-19 rate and the number of airline delays in the U.S between January 2022-July 2022 varies across different airlines.

Questions for Reviewers:
1. How should we best account for Covid test reporting differences between states?
2. How should we best account for differences in airline frequencies across different states when comparing the COVID-19 rates and number of airline delays across different airlines?